# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [188]:
using Manifolds
using Manopt
using LinearAlgebra
using Random

In [189]:
# initialize SPD power manifold P(d)^d1
dd = 3  # size of the SPD matrices, i.e., ∈ R^d×d
M_base = SymmetricPositiveDefinite(dd)
d = manifold_dimension(M_base)
d1 = 100  # size of the signal
M = PowerManifold(M_base, NestedPowerRepresentation(), d1)

PowerManifold(SymmetricPositiveDefinite(3), NestedPowerRepresentation(), 100)

### Generate a data set

In [190]:
e = 1. * Matrix(I, dd, dd)
# compute basis on M_base
Θ = get_basis(M_base, e, DefaultOrthonormalBasis())
#  construct data
Q = fill(e, d1)
# draw random tvectors 
τ = 1.  # variance
σ = .5  # variance
# Xₑⁱ = zeros(d)
# Xₑⁱ[1] = 1.
# Xₑ = get_vector(M_base, e, Xₑⁱ, Θ)
Xₑ = Θ.data[1]

Random.seed!(31)
predata = [exp(M_base, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:d1]

data = [exp(M_base, predata[i], random_tangent(M_base, predata[i], Val(:Gaussian), σ)) for i in 1:d1] # ∈ P(3)^d1
log_Q_data = log(M, Q, data)  # ∈ T_e P(3)^d1

100-element Vector{Matrix{Float64}}:
 [-0.8900780165742711 -0.09498558688520639 0.5136507261426676; -0.09498558688520642 -0.29100843771387586 0.20236196268475953; 0.5136507261426676 0.20236196268475953 -0.8754383868434272]
 [-1.2801403548663093 0.1876986782921049 0.03823688849500963; 0.1876986782921048 -0.8338161834442588 -0.1199456629231202; 0.03823688849500963 -0.1199456629231202 -0.10740799574667402]
 [3.7137293643701934 -0.20520340168201642 0.6033053400508505; -0.20520340168201642 -0.07434699024411584 0.3414838704353225; 0.6033053400508505 0.3414838704353225 -0.48267978013801816]
 [2.2102212734805016 0.08809073063410658 -0.4466734563037301; 0.08809073063410659 0.27452485275248356 -0.10477387695060357; -0.4466734563037302 -0.10477387695060356 0.15300321097265446]
 [0.10988774217278596 0.5959000218193631 0.5425715879939947; 0.5959000218193631 -0.3938676877159909 0.17596994304720495; 0.5425715879939947 0.17596994304720495 -0.017838727102093138]
 [0.16920254237955396 -0.212344514249786

We want to investigate how reliable both ranks are, i.e., we don't want that something looks low-rank, but in reality is higher rank

In [191]:
# compute SVD
# compute Gramm Matrix
Gramm_Q = [inner(M_base, e, log_Q_data[k], log_Q_data[l]) for k=1:d1, l=1:d1]
# compute Σ and V
(sqSigma, V) = eigen(Symmetric(Gramm_Q), d1-min(d1-1,d-1):d1)
# compute U
U = [1/sqrt(sqSigma[i]) * sum([V[k,i] * log_Q_data[k] for k in 1:d1]) for i in 1:d]
# compute low rank approximation of tangent vector
# expoentiate back
# compute error

6-element Vector{Matrix{Float64}}:
 [-0.03770264698847997 -0.39626035223291406 0.17840499702116525; -0.39626035223291395 -0.5084406263071505 -0.4175519173974263; 0.17840499702116522 -0.4175519173974263 0.11690257934779975]
 [0.021517660086877614 0.40770920304799785 0.3013521918053259; 0.40770920304799774 -0.10877305875930907 -0.09856722539272646; 0.3013521918053259 -0.09856722539272643 0.6739395674965789]
 [0.011876873819938965 0.11020543613463071 0.5476199540618792; 0.11020543613463071 0.02123187579937062 0.030378205153147413; 0.5476199540618792 0.030378205153147392 -0.611143822795368]
 [0.02123751041667564 0.008139633125295337 -0.0036072635873610126; 0.00813963312529536 0.7478029883207209 -0.4688375306176127; -0.003607263587361015 -0.4688375306176127 -0.02374587590826708]
 [0.018914157833963312 -0.40525458977280526 0.2780764748097957; -0.40525458977280526 0.41028451788561016 0.3083925458051865; 0.27807647480979564 0.3083925458051865 0.3974686498974033]
 [-0.9985818260750827 0.0175546

In [192]:
function β(κ)
    (κ < 0) && return sinh(sqrt(-κ)) / ( sqrt((-κ)))
    (κ > 0) && return sin(sqrt(κ)) / (sqrt(κ))
    return 1.0 # cuvature zero.
end

β (generic function with 1 method)

In [193]:
function reweighted_inner(M, p, X, Y, g)
    
end

reweighted_inner (generic function with 1 method)

In [194]:
# compute SVD
# we want to compute the metric tensor entries in default ONB at e and then compute g_ij so that we don't have to reevaluate this all the time
gᵢⱼ = zeros(d,d)
for k=1:d1
    Ξₖ = get_basis(M_base, e, DiagonalizingOrthonormalBasis(log_Q_data[k]))
    κₖ = Ξₖ.data.eigenvalues
    βₖ = [β(κₖ[l]) for l in 1:d]
    gᵢⱼ += 1/d1 * [sum([βₖ[l]^2 * inner(M_base, e, Θ.data[i], Ξₖ.data.vectors[l]) * inner(M_base, e, Θ.data[j], Ξₖ.data.vectors[l]) for l=1:d]) for i=1:d, j=1:d]
end
# compute rescaled Gramm Matrix
reweighted_Gramm_Q = [sum([gᵢⱼ[i,j] * get_coordinates(M_base, e, log_Q_data[k], Θ)[i] * get_coordinates(M_base, e, log_Q_data[l], Θ)[j] for i=1:d, j=1:d]) for k=1:d1, l=1:d1]
# compute Σ and V
(reweighted_sqSigma, reweighted_V) = eigen(Symmetric(reweighted_Gramm_Q ), d1-min(d1-1,d-1):d1)
# compute U
reweighted_U = [1/sqrt(reweighted_sqSigma[i]) * sum([reweighted_V[k,i] * log_Q_data[k] for k in 1:d1]) for i in 1:d]
# compute low rank approximation of tangent vector


6-element Vector{Matrix{Float64}}:
 [-0.02998509307768279 -0.016283575328313062 -0.008489281519320166; -0.01628357532831307 -0.593065847525118 -0.46105265141075213; -0.008489281519320165 -0.46105265141075213 0.38006718915452925]
 [-0.05641496902069609 -0.008091218340739954 -0.008652959930814995; -0.00809121834073997 -0.3571657501050422 -0.14754413728847768; -0.008652959930815003 -0.14754413728847768 -0.891459740670021]
 [-0.034582733008840794 -0.012596523229398087 0.01228994072890271; -0.012596523229398101 -0.6856093244882059 0.4800779847875004; 0.012289940728902719 0.4800779847875004 0.100816550770726]
 [0.9712624933835372 -0.02414763903453422 -0.0062139577613103845; -0.02414763903453422 -0.03772945002657839 -0.007823589289988185; -0.006213957761310385 -0.007823589289988185 -0.00930235979369836]
 [-1.8978297339489985e-5 -5.1714513902588385e-6 4.602087415886709e-5; -5.171451390258839e-6 8.09752854293267e-6 4.764856211738944e-6; 4.602087415886709e-5 4.764856211738944e-6 -4.3829007092374

In [208]:
println(sqSigma/sqSigma[end])
renormalized_sqSigma = [reweighted_sqSigma[i] * norm(M_base, e, reweighted_U[i])^2 for i in 1:d]
println(renormalized_sqSigma/renormalized_sqSigma[end])

[0.01605906252986433, 0.019325524430407387, 0.02691785482127655, 0.02903647701733745, 0.031248627171497765, 1.0]
[0.6660954615052969, 0.8782111861513708, 1.039738534798801, 35.62084053103035, 1.0334192079545346, 1.0]


In [206]:
rank = 3
log_Q_data_r = [sum([U[i] * sqrt(sqSigma[i]) * V[k,i] for i in d-rank+1:d]) for k in 1:d1]
reweighted_log_Q_data_r = [sum([reweighted_U[i] * sqrt(reweighted_sqSigma[i]) * reweighted_V[k,i] for i in d-rank+1:d]) for k in 1:d1]
# expoentiate back
data_r = exp(M, Q, log_Q_data_r)
reweighted_data_r = exp(M, Q, reweighted_log_Q_data_r)
# compute error
distance_r = distance(M, data, data_r)
reweighted_distance_r = distance(M, data, reweighted_data_r)

println(distance_r)
println(reweighted_distance_r)

# TODO plot distances for every rank 1:6
# It seems to pick the correct vectors, but the singular values do not yet reflect that well how low the rank actually is...
# -> we can reweight the U's and the sigmas by normalizing them wrt old metric
# the SPD manifold always has a couple of directions with negative curvature, check whether these are the same as the obtained directions

26.302701371271045
9.097659317460128
